# Import / Config

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import Markdown, display

import edurel.utils.dbcon as dbcu
import edurel.utils.db as dbu
import edurel.utils.duckdb as ddbu
import edurel.utils.llm as llmu
import edurel.utils.llmchat as llmc
import edurel.widgets.mermaid_viz as mmw
import edurel.widgets.chatman as cmw
import edurel.utils.misc as mu

load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
DB_DIR = f"{BASE_DIR}/databases"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [31]:
con = dbcu.adw_olap_small()
additional_fks = {}
# additional_fks = {"Orgeinheit": ["Mitarbeiter|Leitung|MID", "Orgeinheit|Obereinheit|OEID"]}

db = dbu.DbHandler(con,additional_fks=additional_fks)
db.schema_yaml_print(["nullable", "fkname"])
db.schema_mermaid_png(output_path="schema.png", omit_tags=["nullable", "fkname"], direction="TB", scale=8)

tables:
- tablename: DimAccount
  columns:
  - columnname: AccountKey
    type: INTEGER
  - columnname: AccountDescription
    type: VARCHAR
  - columnname: AccountType
    type: VARCHAR
  primary_key:
  - AccountKey
- tablename: DimCurrency
  columns:
  - columnname: CurrencyKey
    type: INTEGER
  - columnname: CurrencyName
    type: VARCHAR
  primary_key:
  - CurrencyKey
- tablename: DimCustomer
  columns:
  - columnname: CustomerKey
    type: INTEGER
  - columnname: GeographyKey
    type: INTEGER
  - columnname: FirstName
    type: VARCHAR
  - columnname: LastName
    type: VARCHAR
  - columnname: BirthDate
    type: DATE
  - columnname: MaritalStatus
    type: VARCHAR
  - columnname: Gender
    type: VARCHAR
  - columnname: YearlyIncome
    type: DECIMAL(13,2)
  - columnname: TotalChildren
    type: INTEGER
  - columnname: NumberChildrenAtHome
    type: INTEGER
  - columnname: EnglishEducation
    type: VARCHAR
  - columnname: EnglishOccupation
    type: VARCHAR
  - columnname: Ho

In [9]:
sql = """
select * from FactInternetSales limit 5;
"""
db.sql_print(sql)

┌────────────┬──────────────┬────────────┬─────────────┬─────────────┬──────────────┬─────────────┬───────────────────┬──────────────────┬──────────────────────┬────────────────┬───────────────┬───────────────┬────────────────┬──────────────────────┬────────────────┬─────────────────────┬──────────────────┬───────────────┬───────────────┬───────────────┬───────────────────────┬──────────────────┬───────────┬───────────┬───────────┐
│ ProductKey │ OrderDateKey │ DueDateKey │ ShipDateKey │ CustomerKey │ PromotionKey │ CurrencyKey │ SalesTerritoryKey │ SalesOrderNumber │ SalesOrderLineNumber │ RevisionNumber │ OrderQuantity │   UnitPrice   │ ExtendedAmount │ UnitPriceDiscountPct │ DiscountAmount │ ProductStandardCost │ TotalProductCost │  SalesAmount  │    TaxAmt     │    Freight    │ CarrierTrackingNumber │ CustomerPONumber │ OrderDate │  DueDate  │ ShipDate  │
│   int32    │    int32     │   int32    │    int32    │    int32    │    int32     │    int32    │       int32       │     varc